# University of Michigan Intermediate PostgreSQL Week 2 Exercise

## Unesco Heritage Sites Many-to-One

In this assignment you will read some Unesco Heritage Site data in comma-separated-values (CSV) format and produce properly normalized tables as specified below.

Here is the structure of the tables you will need for this assignment:

```SQL
DROP TABLE IF EXISTS unesco_raw;
CREATE TABLE unesco_raw (
    name TEXT, 
    description TEXT, 
    justification TEXT, 
    year INTEGER,
    longitude FLOAT, 
    latitude FLOAT, 
    area_hectares FLOAT,
    category TEXT, 
    category_id INTEGER, 
    state TEXT, 
    state_id INTEGER,
    region TEXT, 
    region_id INTEGER, 
    iso TEXT, 
    iso_id INTEGER);

CREATE TABLE category (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);
... More tables needed
```

If you run the program multiple times in testing or with different files, make sure to empty out the data before each run.

Your assignment consists of:

1. Loading the <a href='https://www.pg4e.com/tools/sql/whc-sites-2018-small.csv?PHPSESSID=c1287267e05e2987f9ba9211319fb45c%22'>CSV data</a> for this assignment using the following copy command:

```SQL
\copy unesco_raw(name,description,justification,year,longitude,latitude,area_hectares,category,state,region,iso) FROM 'whc-sites-2018-small.csv' WITH DELIMITER ',' CSV HEADER;
```

&ensp;&ensp;Note:</u></b>
- Adding HEADER causes the CSV loader to skip the first line in the CSV file. 
- The `\copy` command must be on <b>one</b> long line.

2. Normalize the data in the <span style='color:green'>unesco_raw table</span> by adding the entries to each of the lookup tables (category, etc.).

3. Add the foreign key columns to the <span style='color:green'>unesco_raw table</span>. 

4. Make a new table called <span style='color:green'>unesco</span> that removes all of the un-normalized redundant text columns like category, etc.

5. To grade this assignment, the auto-grader will run a query like this on your database: 

```SQL
SELECT unesco.name, year, category.name, state.name, region.name, iso.name
  FROM unesco
  JOIN category ON unesco.category_id = category.id
  JOIN iso ON unesco.iso_id = iso.id
  JOIN state ON unesco.state_id = state.id
  JOIN region ON unesco.region_id = region.id
  ORDER BY region.name, unesco.name
  LIMIT 3;
```

The expected result of this query on your database is:

| Name	| Year  | Category | State | Region | iso  |
| :---- | :---- | :----    | :---- | :----  | :----|
| Khomani Cultural Landscape | 2017 | Cultural | South Africa | Africa | za |
| Aapravasi Ghat | 2006 | Cultural | Mauritius | Africa | mu|
| Air and T n r Natural Reserves | 1991 | Natural | Niger | Africa | ne |


This notebook uses both the IPython magic `%sql` and `%%sql` as well as the Psycopg2 DBAPI.  The reason why I chose to use both is because I like the simplicity of the IPython magic but I haven't figured out a way to copy CSV files using the magics.  Thus, I have to rely on Psycopg2's interface for this purpose.  

### Setting Up The Connection

In [1]:
# Import necessary libraries
# courses_db_user_julia contains the PostgreSQL settings as a dictionary file for privacy 
# Import some libraries

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from courses_db_user_julia import postgresql as settings
from pgspecial.main import PGSpecial
import psycopg2 as ps
import pandas as pd

In [2]:
# Get Version

sqlalchemy.__version__ 

'1.4.46'

In [3]:
# Create a get_engine function to get our credentials and create an engine

def get_engine(user, passwd, host, port, db):
    url = f"postgresql://{user}:{passwd}@{host}:{port}/{db}"
    engine = create_engine(url)
    return engine

In [4]:
engine = get_engine(settings['user'],
          settings['password'],
          settings['host'],
          settings['port'],
          settings['dbname'])

In [5]:
# Start Session

session = sessionmaker(bind=engine)()
session

#### IPython Magic!

In [6]:
# Load IPython-SQL module

%load_ext sql

In [7]:
# Create the connection using $
# The $ is a special character in IPython SQL that encloses variables with quotes

%sql $engine.url

In [8]:
# Remove connection display when using magics

%config SqlMagic.displaycon = False

### 0. Create Tables

Per the instructions above, we are to create the following tables:

- unesco_raw
- category
- and more including:
    - state
    - region
    - iso

In [9]:
%%sql

DROP TABLE IF EXISTS unesco_raw;
CREATE TABLE unesco_raw (
    name TEXT, 
    description TEXT, 
    justification TEXT, 
    year INTEGER,
    longitude FLOAT, 
    latitude FLOAT, 
    area_hectares FLOAT,
    category TEXT, 
    category_id INTEGER, 
    state TEXT, 
    state_id INTEGER,
    region TEXT, 
    region_id INTEGER, 
    iso TEXT, 
    iso_id INTEGER);

DROP TABLE IF EXISTS category CASCADE;
CREATE TABLE category (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

DROP TABLE IF EXISTS state CASCADE;
CREATE TABLE state (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

DROP TABLE IF EXISTS region CASCADE;
CREATE TABLE region (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

DROP TABLE IF EXISTS iso CASCADE;
CREATE TABLE iso (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### 1. Load the CSV data for this assignment into the unesco_raw table

Now we need to copy the <span style='color:pink'>unesco_raw data</span> into the <span style='color:green'>unesco_raw table</span>.  This can be done using the `\copy` function in the PostgreSQL psql <b><u>terminal</u></b>.  The reason we use `\copy` as opposed to `COPY` is because `COPY FROM` instructs the PostgreSQL <b>server</b> process to read a file. Whereas the `\copy` is used on the <b>client</b> side.

However, this doesn't seem to work here using IPython magic.  So, we will use Psycopg2.  The basic steps are:

- 1a) Create a Psycopg2 connection object
- 1b) Create a dump file object
- 1c) Create a cursor object
- 1d) Use the `cursor.copy_expert()` function to copy the csv file

#### 1a) Create a Psycopg2 connection object

In [10]:
conn = ps.connect(user=settings['user'],
          password=settings['password'],
          host=settings['host'],
          port=settings['port'],
          dbname=settings['dbname'])

#### 1b) Create a dump file object

In [11]:
# Create a file object where we will dump our csv file to 
# This will allow us to copy the dump into our table

file_obj = open('/mnt/a/docker_share/sql/whc-sites-2018-small.csv', 'r') 

#### 1c) Create a cursor object

Psycopg2 requires the use of a cursor object in order to execute commands to the PostgreSQL server.

In [12]:
cur = conn.cursor()

#### 1d) Use the cursor.copy_expert() function to copy our csv file

In general, the basic syntax is: &ensp;`.copy_expert(sql, file)`

More specifically...

-------------
The basic syntax to copy <b><u>FROM</u></b> a file to a table is: 

(note: STDIN is short for standard input -- STDIN is an input stream where data is sent to and read by a program)

```Python
connection.cursor.copy_expert (
  "COPY table_where_csv_data_goes
  FROM STDIN
  WITH (
    FORMAT CSV,
    DELIMITER ',',
    HEADER
    );",
  file_object
  )
```

--------------
The basic syntax to copy a table <b><u>OUT</u></b> to save a file:

(note: STDOUT is short for standard output -- STDOUT is an output stream where data is sent to and read by a program)

```Python
connection.cursor.copy_expert (
  "COPY table_to_save
  TO STDOUT
  WITH (
    FORMAT CSV,
    DELIMITER ',',
    HEADER
);",
  file_object
  )
```

<p style='color:orange'>Change the format, delimiter and/or header as required.</p>

#### 1d) Use the `cursor.copy_expert()` function to copy the csv file

In [13]:
# Check if there is any data in the unesco_raw table -- there shouldn't be 

%sql SELECT * FROM unesco_raw LIMIT 1;

0 rows affected.


name,description,justification,year,longitude,latitude,area_hectares,category,category_id,state,state_id,region,region_id,iso,iso_id


In [14]:
# Copy the data from the input stream to the track_raw table

cur.copy_expert("COPY unesco_raw(name,description,justification,year,longitude,latitude,area_hectares,category,state,region,iso) FROM STDIN WITH (FORMAT CSV, DELIMITER ',', HEADER);", file_obj)

<p><span style='color:red; font-size:20px'>&#x26A0;&ensp;Error?</span> &ensp;&ensp;&ensp; You need to rollback the command or nothing else will execute.</p>

In [ ]:
# If you make a mistake or there is an error issue this command

conn.rollback()

<span style='color:green; font-size:20px'>&#x2705;&ensp;Everything OK?</span>&ensp;&ensp;&ensp; Commit!

In [15]:
# If everything works and you want it to stick issue this command

conn.commit()

In [16]:
# Test to make sure everything worked

%sql SELECT name, year, category, state, region, iso FROM unesco_raw LIMIT 3;

3 rows affected.


name,year,category,state,region,iso
Cultural Landscape and Archaeological Remains of the Bamiyan Valley,2003,Cultural,Afghanistan,Asia and the Pacific,af
Minaret and Archaeological Remains of Jam,2002,Cultural,Afghanistan,Asia and the Pacific,af
Historic Centres of Berat and Gjirokastra,2005,Cultural,Albania,Europe and North America,al


Just as an FYI, it's also possible to use Psycopg2 to run other SQL commands.  This is done by the `cursor.execute()` function.  Multiple SQL commands can be sent at once.

For example:

```Python
cursor.execute ("""
    
DROP TABLE IF EXISTS unesco_raw;

CREATE TABLE unesco_raw
 (name TEXT, description TEXT, justification TEXT, year INTEGER,
    longitude FLOAT, latitude FLOAT, area_hectares FLOAT,
    category TEXT, category_id INTEGER, state TEXT, state_id INTEGER,
    region TEXT, region_id INTEGER, iso TEXT, iso_id INTEGER);
""")
```


### 2. Normalize the data in the unesco_raw table by adding the entries to each of the lookup tables (category, etc.).

First, recall the schemas:

In [17]:
%%sql category_schema <<

SELECT 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'category';

2 rows affected.
Returning data to local variable category_schema


In [18]:
%%sql state_schema <<

SELECT 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'state';

2 rows affected.
Returning data to local variable state_schema


In [19]:
%%sql region_schema <<

SELECT 
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'region';

2 rows affected.
Returning data to local variable region_schema


In [20]:
%%sql iso_schema <<  
    
SELECT  
   column_name, 
   data_type 
FROM 
   information_schema.columns
WHERE 
   table_name = 'iso';

2 rows affected.
Returning data to local variable iso_schema


In [21]:
print('Category Schema:\n {} \n State Schema\n: {} \n Region Schema\n: {} \n iso Schema\n: {}'.format(category_schema, state_schema, region_schema, iso_schema))

Category Schema:
 +-------------+-------------------+
| column_name |     data_type     |
+-------------+-------------------+
|      id     |      integer      |
|     name    | character varying |
+-------------+-------------------+ 
 State Schema
: +-------------+-------------------+
| column_name |     data_type     |
+-------------+-------------------+
|      id     |      integer      |
|     name    | character varying |
+-------------+-------------------+ 
 Region Schema
: +-------------+-------------------+
| column_name |     data_type     |
+-------------+-------------------+
|      id     |      integer      |
|     name    | character varying |
+-------------+-------------------+ 
 iso Schema
: +-------------+-------------------+
| column_name |     data_type     |
+-------------+-------------------+
|      id     |      integer      |
|     name    | character varying |
+-------------+-------------------+


In [22]:
%%sql

INSERT INTO category (name) SELECT DISTINCT category FROM unesco_raw;
INSERT INTO state (name) SELECT DISTINCT state FROM unesco_raw;
INSERT INTO region (name) SELECT DISTINCT region FROM unesco_raw;
INSERT INTO iso (name) SELECT DISTINCT iso FROM unesco_raw;

3 rows affected.
163 rows affected.
5 rows affected.
163 rows affected.


[]

Verify it worked:

In [23]:
%sql category_name << SELECT * FROM category LIMIT 1;
%sql state_name << SELECT * FROM state LIMIT 1;
%sql region_name << SELECT * FROM region LIMIT 1;
%sql iso_name << SELECT * FROM iso LIMIT 1;   

1 rows affected.
Returning data to local variable category_name
1 rows affected.
Returning data to local variable state_name
1 rows affected.
Returning data to local variable region_name
1 rows affected.
Returning data to local variable iso_name


In [24]:
 print('Category:\n {} \n State:\n {} \n Region:\n {} \n iso:\n {}'.format(category_name, state_name, region_name, iso_name))

Category:
 +----+-------+
| id |  name |
+----+-------+
| 1  | Mixed |
+----+-------+ 
 State:
 +----+-----------+
| id |    name   |
+----+-----------+
| 1  | Indonesia |
+----+-----------+ 
 Region:
 +----+----------------------+
| id |         name         |
+----+----------------------+
| 1  | Asia and the Pacific |
+----+----------------------+ 
 iso:
 +----+------+
| id | name |
+----+------+
| 1  |  sm  |
+----+------+


### 3. Add the foreign key columns to the unesco_raw table

Recall, the foreign keys are category_id, state_id, region_id & iso_id

In [25]:
%%sql

UPDATE unesco_raw 
SET 
category_id = (SELECT category.id FROM category WHERE category.name = unesco_raw.category),
state_id = (SELECT state.id FROM state WHERE state.name = unesco_raw.state),
region_id = (SELECT region.id FROM region WHERE region.name = unesco_raw.region),
iso_id = (SELECT iso.id FROM iso WHERE iso.name = unesco_raw.iso);

-- Verify
SELECT category_id, state_id, region_id, iso_id FROM unesco_raw LIMIT 1;

1044 rows affected.
1 rows affected.


category_id,state_id,region_id,iso_id
3,27,5,70


### 4. Make a new table called unesco that removes all of the un-normalized redundant text columns like category, etc.

We first need to create the table schema:

In [26]:
%%sql

CREATE TABLE IF NOT EXISTS unesco (
  id SERIAL,
  name TEXT,
  description TEXT,
  justification TEXT, 
  year INTEGER,
  longitude FLOAT, 
  latitude FLOAT, 
  area_hectares FLOAT,
  category_id INT,
  state_id INT,
  region_id INT,
  iso_id INT
);

Done.


[]

Now we can insert the necessary data from the <span style='color:green'>unesco_raw</sapn> table:

In [27]:
%%sql

INSERT INTO unesco (
  name,
  description,
  justification, 
  year,
  longitude, 
  latitude, 
  area_hectares,
  category_id,
  state_id,
  region_id,
  iso_id)
SELECT 
  name,
  description,
  justification, 
  year,
  longitude, 
  latitude, 
  area_hectares,
  category_id,
  state_id,
  region_id,
  iso_id
FROM
  unesco_raw;

1044 rows affected.


[]

### 5. To grade this assignment, the auto-grader will run a query like this on your database:
 
```SQL
SELECT unesco.name, year, category.name, state.name, region.name, iso.name
  FROM unesco
  JOIN category ON unesco.category_id = category.id
  JOIN iso ON unesco.iso_id = iso.id
  JOIN state ON unesco.state_id = state.id
  JOIN region ON unesco.region_id = region.id
  ORDER BY region.name, unesco.name
  LIMIT 3;
```

The expected result of this query on your database is:

| Name	| Year  | Category | State | Region | iso  |
| :---- | :---- | :----    | :---- | :----  | :----|
| Khomani Cultural Landscape | 2017 | Cultural | South Africa | Africa | za |
| Aapravasi Ghat | 2006 | Cultural | Mauritius | Africa | mu|
| Air and T n r Natural Reserves | 1991 | Natural | Niger | Africa | ne |


So, let's check our work (note: the AS "..." is used to change the column names & capitalize them to match the assignment):

In [28]:
%%sql

SELECT unesco.name AS "Name", year AS "Year", category.name AS "Category", state.name AS "State", region.name AS "Region", iso.name AS iso
  FROM unesco
  JOIN category ON unesco.category_id = category.id
  JOIN iso ON unesco.iso_id = iso.id
  JOIN state ON unesco.state_id = state.id
  JOIN region ON unesco.region_id = region.id
  ORDER BY region.name, unesco.name
  LIMIT 3;

3 rows affected.


Name,Year,Category,State,Region,iso
Khomani Cultural Landscape,2017,Cultural,South Africa,Africa,za
Khomani Cultural Landscape,2017,Cultural,South Africa,Africa,za
Aapravasi Ghat,2006,Cultural,Mauritius,Africa,mu


<span style='color:green; font-size:20px'>&#x2705;&ensp;Success!&ensp;&#x1F389;</span>

<p style='color:red; font-size:22px'>Make sure you CLOSE all connections once you're done:</p>

In [29]:
magic_connections = %sql -l
[c.session.close() for c in magic_connections.values()]

[None]

In [30]:
session.close()
engine = session.get_bind()
engine.dispose() 
file_obj.close()
cur.close()
conn.close()